In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics import r2_score
import sys
import os

In [2]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

from causalnex.structure import StructureModel
from causalnex.structure.notears import from_pandas

In [3]:
sm = StructureModel()

In [4]:
data = pd.read_csv('../test_tables/BD_Demographics_Database_BSI_WhoQol.csv', sep = ',',encoding='latin-1')

In [5]:
new_feat = []#list(data.columns)[42:141]
new_feat += ['WHOQoL_1','WHOQoL_23','WHOQoL_25','WHOQoL_19','WHOQoL_7','WHOQoL_11',
            'WHOQoL_9','WHOQoL_8','WHOQoL_2','WHOQoL_2','WHOQoL_6', 'BSI_9', 'BSI_16', 'BSI_17','BSI_18','BSI_35','BSI_50']

In [6]:
result_cols = []
for col in data.columns:
    if 'whoqol' in col:
        result_cols.append(col)
    elif 'BSI_' in col and len(col)>6:
        result_cols.append(col)
bsi = []
whoqol = []
for col in data.columns:
    if 'WQOL' in col:
        whoqol.append(col)
    elif 'BSI_' in col and len(col)>6:
        bsi.append(col)
        
bsi_feat = []
whoqol_feat = []
for col in data.columns:
    if 'WHOQoL' in col:
        whoqol_feat.append(col)
    elif 'BSI_' in col and len(col)<=6:
        bsi_feat.append(col)

In [7]:

data[result_cols] = data[result_cols].replace(' ', np.nan)
data = data.replace(' ', np.nan)

In [8]:
#data.drop(['ID_covid', 'sample', 'date_created','consent', 'sgID', 'date_birth', 'region','city_br','Occupation_other', 'working_hours','religion_other', 'cceb_class',
#'social_support_n_family','social_support_n_friends'],axis = 1, inplace = True)

In [9]:
#data[bsi] = data[bsi].apply(lambda x: x.str.replace(',','.'))
#data[whoqol] = data[whoqol].apply(lambda x: x.str.replace(',','.'))

In [10]:
data[bsi_feat] = data[bsi_feat].astype(float)

data[whoqol_feat] = data[whoqol_feat].astype(float)

In [11]:
for col in data.columns:
    try:
        data[col] = data[col].astype(float)
    except:
        (print(col))

database
sex
education
marital_status
ethnicity
cceb_class


In [12]:
for col in data.columns:
    if data[col].nunique()==1:
        data.drop(col,inplace = True,axis=1)

In [13]:
whoqol

['WQOL.Fisico', 'WQOL.Psicologico', 'WQOL.RSociais', 'WQOL_MAmbiente']

In [14]:
whoqol_feat_filtrada = set(whoqol_feat) - set(['WHOQoL_3','WHOQoL_4','WHOQoL_10','WHOQoL_15','WHOQoL_16','WHOQoL_17','WHOQoL_18'])

In [15]:
corr = data.corr()[['BSI_2','BSI_7','BSI_23','BSI_29','BSI_30','BSI_33','BSI_37']]
corr.loc['BSI_Somatization_EB']

BSI_2     0.648026
BSI_7     0.709963
BSI_23    0.718463
BSI_29    0.706837
BSI_30    0.695902
BSI_33    0.697546
BSI_37    0.768539
Name: BSI_Somatization_EB, dtype: float64

In [16]:
corr = data.corr()[['WHOQoL_20','WHOQoL_21','WHOQoL_22']]
corr.loc['WQOL.RSociais']

WHOQoL_20    0.821576
WHOQoL_21    0.807779
WHOQoL_22    0.783711
Name: WQOL.RSociais, dtype: float64

In [17]:
corr = data.corr()[['WHOQoL_5','WHOQoL_6','WHOQoL_7','WHOQoL_11','WHOQoL_19','WHOQoL_26']]
corr.loc['WQOL.Psicologico']

WHOQoL_5     0.700694
WHOQoL_6     0.728614
WHOQoL_7     0.705583
WHOQoL_11    0.737199
WHOQoL_19    0.825160
WHOQoL_26   -0.728383
Name: WQOL.Psicologico, dtype: float64

In [18]:
bsi

['BSI_GSI_EB',
 'BSI_Somatization_EB',
 'BSI_ObsCom_EB',
 'BSI_IntSens_EB',
 'BSI_Depression_EB',
 'BSI_Anxiety_EB',
 'BSI_Hostility_EB',
 'BSI_Phobic_EB',
 'BSI_Paranoid_EB',
 'BSI_Psychoticism_EB']

In [19]:
data.fillna(-1, inplace = True)
Y1 = data['WQOL_MAmbiente']
Y2 = data[whoqol]
#data['mean'] = np.round(data['mean']/10)

#Y2 = data['"ARTERIAL_DIAS"']
#data.drop(whoqol + bsi ,axis = 1, inplace = True)
X = data[  list(whoqol_feat)  + ['WQOL_MAmbiente']]
#X = data[['BSI_9', 'BSI_16', 'BSI_17','BSI_18','BSI_35','BSI_50', 'BSI_Somatization_EB','BSI_1','BSI_12',
#             'BSI_19','BSI_38','BSI_45','BSI_49', 'BSI_2','BSI_7','BSI_23','BSI_29','BSI_30','BSI_33','BSI_37']]
X2 = data[new_feat]

In [20]:
X.to_csv('../test_tables/covid_reduced_WQOLenv_full.csv',index=False,sep=',')